## MAKİNE ÖĞRENMESİNE GİRİŞ
### DOĞRUSAL OLMAYAN REGRESYON MODELİ- YAPAY SİNİR AĞLARI


In [62]:
import warnings 
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


In [63]:
df=pd.read_csv("/Users/bernaersari/Downloads/Hitters.csv")

In [64]:
df # beyzbol oyuncularının çeşitli özeLliklerine göre ücretini belirleyen bir model kurmak istiyoruz.

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,497,127,7,65,48,37,5,2703,806,32,379,311,138,N,E,325,9,3,700.0,N
318,492,136,5,76,50,94,12,5511,1511,39,897,451,875,A,E,313,381,20,875.0,A
319,475,126,3,61,43,52,6,1700,433,7,217,93,146,A,W,37,113,7,385.0,A
320,573,144,9,85,60,78,8,3198,857,97,470,420,332,A,E,1314,131,12,960.0,A


In [65]:
df=df.dropna() #boş değerleri silelim.

In [66]:
df.select_dtypes(include='object').columns

Index(['League', 'Division', 'NewLeague'], dtype='object')

In [67]:
dms=pd.get_dummies(df[['League', 'Division', 'NewLeague']]) # one-hot encoding ile kategorik değişkenleri 0-1 değişkenlere dönüştürelim.

In [68]:
y=df['Salary'] #bulmak istediğimiz hedef değişkenimiz olan 'Salary' bağımlı değişkenimizi y olarak tanımlayalım.

In [70]:
X_=df.drop(['Salary','League', 'Division', 'NewLeague'], axis=1) 
#dataframeden encoding yapılan değişkenleri ve bağımlı değişkeni silip dataframedeki verileri X_ olarak tanımlayalım.

In [71]:
X_ #bağımsız değişkenlere göz atalım.

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,632,43,10
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,880,82,14
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,200,11,3
4,321,87,10,39,42,30,2,396,101,12,48,46,33,805,40,4
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,282,421,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,497,127,7,65,48,37,5,2703,806,32,379,311,138,325,9,3
318,492,136,5,76,50,94,12,5511,1511,39,897,451,875,313,381,20
319,475,126,3,61,43,52,6,1700,433,7,217,93,146,37,113,7
320,573,144,9,85,60,78,8,3198,857,97,470,420,332,1314,131,12


In [103]:
X=pd.concat([X_,dms[['League_N', 'Division_W','NewLeague_N']]],axis=1) 
#son olarak tüm bağımsız değişkenleri bir araya getirelim. 
#Dummy değişkenler aynı bilgileri içerdiğinden her birinden bir tane değişkeni bağımsız değişken grubuna ekliyoruz.

In [104]:
X_train,X_test,y_train,y_test=train_test_split(X,
                                               y,
                                               test_size=0.25,
                                               random_state=42) # veri setinin yüzde 25 test seti olarak ayrıldı.


[CV] END .............alpha=0.1, hidden_layer_sizes=(10, 20); total time=   0.1s
[CV] END ...............alpha=0.1, hidden_layer_sizes=(5, 5); total time=   0.1s
[CV] END ...........alpha=0.1, hidden_layer_sizes=(100, 200); total time=   0.7s
[CV] END ............alpha=0.01, hidden_layer_sizes=(10, 20); total time=   0.1s
[CV] END ............alpha=0.01, hidden_layer_sizes=(10, 20); total time=   0.1s
[CV] END ..............alpha=0.01, hidden_layer_sizes=(5, 5); total time=   0.1s
[CV] END ..............alpha=0.01, hidden_layer_sizes=(5, 5); total time=   0.1s
[CV] END ..........alpha=0.01, hidden_layer_sizes=(100, 200); total time=   0.7s
[CV] END ..........alpha=0.01, hidden_layer_sizes=(100, 200); total time=   0.7s
[CV] END ..........alpha=0.02, hidden_layer_sizes=(100, 200); total time=   0.6s
[CV] END ...........alpha=0.001, hidden_layer_sizes=(10, 20); total time=   0.1s
[CV] END .............alpha=0.001, hidden_layer_sizes=(5, 5); total time=   0.1s
[CV] END .............alpha=

__MODEL & TAHMİN__

In [74]:
#yapay sinir ağları homojen veri setinde daha başarılı olduğundan dolayı, standartlaştırma yapılması iyi olur. Standartlaştırma = ölçeklendirme
#scaler isimli bir değişken oluşturup, StandardScaler() fonksiyonunu bu değişkene atayalım.

In [105]:
scaler=StandardScaler()

In [106]:
scaler.fit(X_train)
scaler.fit(X_test)

StandardScaler()

In [107]:
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [108]:
mlp_model=MLPRegressor().fit(X_train_scaled,y_train)

In [109]:
y_pred=mlp_model.predict(X_test_scaled)

In [110]:
test_hatasi=np.sqrt(mean_squared_error(y_test,y_pred))

In [111]:
test_hatasi

660.3108673100032

__MODEL TUNING__

In [112]:
mlp_params={"alpha":[0.1,0.01,0.02,0.001],
           "hidden_layer_sizes":[(10,20),(5,5),(100,200)]}

In [113]:
mlp_cv_model= GridSearchCV(mlp_model, mlp_params, cv=10, verbose=2, n_jobs=-1).fit(X_train_scaled,y_train)

Fitting 10 folds for each of 12 candidates, totalling 120 fits
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require In

/Users/bernaersari/anaconda3/envs/pythonProject1/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/bernaersari/anaconda3/envs/pythonProject1/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/bernaersari/anaconda3/envs/pythonProject1/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/bernaersari/anaconda3/envs/pythonProject1/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and 

In [114]:
best_param=mlp_cv_model.best_params_
best_param # en iyi parametreleri bulalım.

{'alpha': 0.001, 'hidden_layer_sizes': (100, 200)}

In [115]:
mlp_tuned=MLPRegressor(alpha= 0.001, hidden_layer_sizes= (100, 200)).fit(X_train_scaled,y_train)

In [116]:
y_pred=mlp_tuned.predict(X_test_scaled)

In [117]:
np.sqrt(mean_squared_error(y_test,y_pred)) #model_tuning yapmadan önce 660 olan test hatasının 354 olduğunu görüyoruz.

354.09368269951705